In [54]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
import sys
sys.path.append("..")
#import os
from utils.model import Net_mask
import torch
#import numpy as np
import matplotlib.pyplot as plt
from torch.distributions.multivariate_normal import MultivariateNormal

In [56]:
model_ML_inf = Net_mask()
model_MAP_e4 = Net_mask()
model_MAP_e4_R100 = Net_mask()
model_MAP_e5 = Net_mask()
model_MAP_e6 = Net_mask()
model_MAP_e6_R100 = Net_mask()
model_ML_inf_reg = Net_mask()
model_ML_inf_reg_10 = Net_mask()

#model_ML_inf.load_state_dict(torch.load(f'../saved_models/models_infinite_data/ML/model_weights.pth'))
#model_MAP_e4.load_state_dict(torch.load(f'../saved_models/finite_long_e4/map/0/model_weights.pth'))
#model_MAP_e4_R100.load_state_dict(torch.load(f'../saved_models/finite_long_e4_100/map/0/model_weights.pth'))
#model_MAP_e5.load_state_dict(torch.load(f'../saved_models/finite_long_e5/map/0/model_weights.pth'))
#model_MAP_e6.load_state_dict(torch.load(f'../saved_models/finite_long_e6/map/0/model_weights.pth'))
#model_MAP_e6_R100.load_state_dict(torch.load(f'../saved_models/finite_long_e6_100/map/0/model_weights.pth'))
#model_ML_inf_reg.load_state_dict(torch.load(f'../saved_models/models_infinite_data/MAP_inf_prior_100/model_weights.pth'))
#model_ML_inf_reg_10.load_state_dict(torch.load(f'../saved_models/models_infinite_data/MAP_inf_prior_10/model_weights.pth'))


model_ML_inf.load_state_dict(torch.load(f'../saved_models/models_infinite_data/ML/model_weights.pth'))
model_ML_inf_reg.load_state_dict(torch.load(f'../saved_models/models_infinite_data/MAP_inf_prior_100/model_weights.pth'))
model_ML_inf_reg_10.load_state_dict(torch.load(f'../saved_models/models_infinite_data/MAP_inf_prior_10/model_weights.pth'))
model_MAP_e4.load_state_dict(torch.load(f'../saved_models/models_finite_data/e4/prior_10/model_weights.pth'))
model_MAP_e4_R100.load_state_dict(torch.load(f'../saved_models/models_finite_data/e4/prior_100/model_weights.pth'))
model_MAP_e5.load_state_dict(torch.load(f'../saved_models/models_finite_data/e5/prior_100/model_weights.pth'))
model_MAP_e6.load_state_dict(torch.load(f'../saved_models/models_finite_data/e6/prior_1000/model_weights.pth'))
model_MAP_e6_R100.load_state_dict(torch.load(f'../saved_models/models_finite_data/e6/prior_100/model_weights.pth'))

<All keys matched successfully>

In [57]:
tensor_D = torch.tensor(
    [[1, -1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, -1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0,  1, -1, 0, 0, 0, 0, 0, 0],
    [0, 0,  0, 1, -1, 0, 0, 0, 0, 0],
    [0, 0,  0, 0, 1, -1, 0, 0, 0, 0],
    [0, 0,  0, 0, 0, 1, -1, 0, 0, 0],
    [0, 0,  0, 0, 0, 0, 1, -1, 0, 0],
    [0, 0,  0, 0, 0, 0, 0, 1, -1, 0],
    [0, 0,  0, 0, 0, 0, 0, 0, 1, -1],
    [-1, 0, 0, 0, 0, 0, 0, 0, 0, 1],], dtype=torch.float
)
tensor_Q_m  = torch.mm(torch.t(tensor_D), tensor_D)
n_param = tensor_D.size(dim=0)
#tau2 = 1000 # 1/tau2 is the noise added to the diagonal
#tensor_Q_m_modified = tensor_Q_m + torch.eye(n_param)*(1./tau2)
#sigma2_eps = 0.01  # 1/sigma2_eps is the factor before the likelihood
#tensor_mu_m = torch.zeros(n_param)

#tensor_Sigma_m = torch.inverse(tensor_Q_m_modified)
#tensor_Sigma_eps = torch.eye(n_param)*sigma2_eps
#tensor_mu_eps = tensor_mu_m 

In [58]:
all_test_obses = torch.load(f'../data/test_observations/test_set_full/all_test_obses.pt')
all_test_solutions = torch.load(f'../data/test_observations/test_set_full/all_test_solutions.pt')

In [59]:
def loss(predicted, analytical):
    difference = predicted - analytical
    squared_error = torch.square(difference)
    tensor_squared_error_sum = torch.sum(squared_error, dim=1)
    tensor_squared_error_sum_all = torch.sum(tensor_squared_error_sum)
    return tensor_squared_error_sum_all

In [60]:
max_points = 6
n_bnn = 100
n_data = all_test_obses.shape[1]

loss_ML_inf = torch.zeros(max_points)
loss_ML_inf_reg = torch.zeros(max_points)
loss_ML_inf_reg_10 = torch.zeros(max_points)
loss_MAP_e4 = torch.zeros(max_points)
loss_MAP_e4_R100 = torch.zeros(max_points)
loss_MAP_e5 = torch.zeros(max_points)
loss_MAP_e6 = torch.zeros(max_points)
loss_MAP_e6_R100 = torch.zeros(max_points)

loss_RML_e4_0 = torch.zeros(max_points)
loss_RML_e4_mean = torch.zeros(max_points)
loss_RML_e4_median = torch.zeros(max_points)
loss_RML_e4_variance = torch.zeros(max_points)
loss_RML_e5_0 = torch.zeros(max_points)
loss_RML_e5_mean = torch.zeros(max_points)
loss_RML_e5_median = torch.zeros(max_points)
loss_RML_e5_variance = torch.zeros(max_points)

for p in range(max_points):
    test_set_points = all_test_obses[p]
    test_set_solution = all_test_solutions[p]

    n_bnn = 43
    pred_RML_e4_all = torch.zeros(n_bnn, n_data, n_param)
    pred_RML_e5_all = torch.zeros(n_bnn, n_data, n_param)
    i_list = list(range(26)) + list(range(83, 100))
    for idx, item in enumerate(i_list):
        model_RML_e4 = Net_mask()
        model_RML_e4.load_state_dict(torch.load(f'../saved_models/rml/long/e4/pos/{item}/model_weights.pth'))
        pred_RML_e4_all[idx] = model_RML_e4(test_set_points)
    i_list = list(range(35)) + list(range(92, 100))
    for idx, item in enumerate(i_list):
        model_RML_e5 = Net_mask()
        model_RML_e5.load_state_dict(torch.load(f'../saved_models/rml/long/e5/pos/{item}/model_weights.pth'))
        pred_RML_e5_all[idx] = model_RML_e5(test_set_points)

    loss_ML_inf[p] = loss(model_ML_inf(test_set_points), test_set_solution)
    loss_ML_inf_reg[p] = loss(model_ML_inf_reg(test_set_points), test_set_solution)
    loss_ML_inf_reg_10[p] = loss(model_ML_inf_reg_10(test_set_points), test_set_solution)
    loss_MAP_e4[p] = loss(model_MAP_e4(test_set_points), test_set_solution)
    loss_MAP_e4_R100[p] = loss(model_MAP_e4_R100(test_set_points), test_set_solution)
    loss_MAP_e5[p] = loss(model_MAP_e5(test_set_points), test_set_solution)
    loss_MAP_e6[p] = loss(model_MAP_e6(test_set_points), test_set_solution)
    loss_MAP_e6_R100[p] = loss(model_MAP_e6_R100(test_set_points), test_set_solution)

    loss_RML_e4_0[p] = loss(pred_RML_e4_all[0], test_set_solution)
    loss_RML_e4_mean[p] = loss(torch.mean(pred_RML_e4_all, dim=0), test_set_solution)
    loss_RML_e4_median[p] = loss(torch.median(pred_RML_e4_all, dim=0)[0], test_set_solution)
    loss_RML_e4_variance[p] = torch.mean(torch.var(pred_RML_e4_all, dim=0))

    loss_RML_e5_0[p] = loss(pred_RML_e5_all[0], test_set_solution)
    loss_RML_e5_mean[p] = loss(torch.mean(pred_RML_e5_all, dim=0), test_set_solution)
    loss_RML_e5_median[p] = loss(torch.median(pred_RML_e5_all, dim=0)[0], test_set_solution)
    loss_RML_e5_variance[p] = torch.mean(torch.var(pred_RML_e5_all, dim=0))

In [61]:
title = f'{"Model/statistic":15} | {"1 point":8} | {"2 points":8} | {"3 points":8} | {"4 points":8} | {"5 points":8} | {"6 points":8} |'

string_ML_inf = f'{"ML-inf":15} |'
#string_ML_inf_reg = f'{"ML-inf-reg":15} | {"inf":4} |'
string_ML_inf_reg = f'{"MAP-inf":15} |'
string_ML_inf_reg_10 = f'{"ML-inf-reg_10":15} |'

string_MAP_e4 = f'{"MAP-e4":15} |'
string_MAP_e4_R100 = f'{"MAP-e4":15} |'
string_MAP_e5 = f'{"MAP-e5":15} |'
string_MAP_e6 = f'{"MAP-e6":15} |'
string_MAP_e6_R100 = f'{"MAP-e6":15} |'

string_RML_e4_0 = f'{"RML-e4-1":15} |'
string_RML_e4_mean = f'{"RML-e4-mean":15} |'
string_RML_e4_median = f'{"RML-e4-median":15} |'

string_RML_e5_0 = f'{"RML-e5-1":15} |'
string_RML_e5_mean = f'{"RML-e5-mean":15} |'
string_RML_e5_median = f'{"RML-e5-median":15} |'

#string_RML_e6_0 = f'{"RML-e6-0":15} | {"1e6":4} |'
#string_RML_e6_mean = f'{"RML-e6-mean":15} | {"1e6":4} |'
#string_RML_e6_median = f'{"RML-e6-median":15} | {"1e6":4} |'

string_RML_e4_var = f'{"RML-e4-var":15} |'
string_RML_e5_var = f'{"RML-e5-var":15} |'


for i in range(max_points):

    string_ML_inf += f' {loss_ML_inf[i]:8.2f} |' 
    string_ML_inf_reg += f' {loss_ML_inf_reg[i]:8.2f} |' 
    string_ML_inf_reg_10 += f' {loss_ML_inf_reg_10[i]:8.2f} |' 

    string_MAP_e4 += f' {loss_MAP_e4[i]:8.2f} |' 
    string_MAP_e4_R100 += f' {loss_MAP_e4_R100[i]:8.2f} |' 
    string_MAP_e5 += f' {loss_MAP_e5[i]:8.2f} |' 
    string_MAP_e6 += f' {loss_MAP_e6[i]:8.2f} |' 
    string_MAP_e6_R100 += f' {loss_MAP_e6_R100[i]:8.2f} |' 

    string_RML_e4_0 += f' {loss_RML_e4_0[i]:8.2f} |' 
    string_RML_e4_mean += f' {loss_RML_e4_mean[i]:8.2f} |' 
    string_RML_e4_median += f' {loss_RML_e4_median[i]:8.2f} |' 

    string_RML_e5_0 += f' {loss_RML_e5_0[i]:8.2f} |' 
    string_RML_e5_mean += f' {loss_RML_e5_mean[i]:8.2f} |' 
    string_RML_e5_median += f' {loss_RML_e5_median[i]:8.2f} |' 

    string_RML_e4_var += f' {loss_RML_e4_variance[i]:8.4f} |' 
    string_RML_e5_var += f' {loss_RML_e5_variance[i]:8.4f} |' 


string_break = '-'*83
total_string = ""

print(title); total_string+=title+'\n'
print(string_break); total_string+=string_break+'\n'
print(string_ML_inf); total_string+=string_ML_inf+'\n'
print(string_ML_inf_reg); total_string+=string_ML_inf_reg+'\n'
#print(string_ML_inf_reg_10); total_string+=string_ML_inf_reg_10+'\n'
print(string_break); total_string+=string_break+'\n'

#print(string_MAP_e4); total_string+=string_MAP_e4+'\n'
print(string_MAP_e4_R100); total_string+=string_MAP_e4_R100+'\n'
print(string_MAP_e5); total_string+=string_MAP_e5+'\n'
#print(string_MAP_e6); total_string+=string_MAP_e6+'\n'
print(string_MAP_e6_R100); total_string+=string_MAP_e6_R100+'\n'
print(string_break); total_string+=string_break+'\n'

print(string_RML_e4_0); total_string+=string_RML_e4_0+'\n'
print(string_RML_e4_mean); total_string+=string_RML_e4_mean+'\n'
print(string_RML_e4_median); total_string+=string_RML_e4_median+'\n'
print(string_break); total_string+=string_break+'\n'

print(string_RML_e5_0); total_string+=string_RML_e5_0+'\n'
print(string_RML_e5_mean); total_string+=string_RML_e5_mean+'\n'
print(string_RML_e5_median); total_string+=string_RML_e5_median+'\n'
print(string_break); total_string+=string_break+'\n'

print(string_RML_e4_var); total_string+=string_RML_e4_var+'\n'
print(string_RML_e5_var); total_string+=string_RML_e5_var+'\n'
print(string_break); total_string+=string_break+'\n'


Model/statistic | 1 point  | 2 points | 3 points | 4 points | 5 points | 6 points |
-----------------------------------------------------------------------------------
ML-inf          |   625.19 |    65.79 |   192.65 |   144.46 |    77.31 |    37.18 |
MAP-inf         |   443.55 |    41.70 |   150.94 |   100.98 |    47.48 |    22.96 |
-----------------------------------------------------------------------------------
MAP-e4          |  1088.65 |   179.28 |   198.51 |   142.28 |    77.93 |    39.00 |
MAP-e5          |   644.86 |    95.18 |   210.09 |   153.28 |    75.92 |    34.89 |
MAP-e6          |   663.73 |    81.26 |   196.81 |   154.13 |    79.78 |    39.50 |
-----------------------------------------------------------------------------------
RML-e4-1        |   863.98 |   100.56 |   169.18 |   123.36 |    64.08 |    32.51 |
RML-e4-mean     |   973.23 |    90.41 |   160.36 |   114.73 |    55.44 |    21.92 |
RML-e4-median   |   866.27 |    82.40 |   159.82 |   114.71 |    55.63 |    

In [62]:
with open('../figures/plot_table_prediction_error/plot_table_prediction_error.txt', 'w') as outfile:
    outfile.write('\n\n' + total_string[:-1])

In [63]:
title1 = f'{"Model":15} | {"N train data":12} | {"Regularization":14} | {"Comment":34} |'
title2 = f'{"Statistic":15} | {"":12} | {"":14} | {"Comment":34} |'

string_ML_inf = f'{"ML-inf":15} | {"Infinite":12} | {"None":14} | {"Maximum likelihood model":34} |'
string_ML_inf_reg = f'{"MAP-inf":15} | {"Infinite":12} | {"L_2":14} | {"Model with L_2 regularization":34} |'

#string_MAP_e4 = f'{"MAP-e4":13} | {"1e4":4} |'
string_MAP_e4_R100 = f'{"MAP-e4":15} | {"10^4":12} | {"L_2":14} | {"L_2 reg. trained with 10^4 data":34} |'
string_MAP_e5 = f'{"MAP-e5":15} | {"10^5":12} | {"L_2":14} | {"L_2 reg. trained with 10^5 data":34} |'
#string_MAP_e6 = f'{"MAP-e6":13} | {"1e6":4} |'
string_MAP_e6_R100 = f'{"MAP-e6":15} | {"10^6":12} | {"L_2":14} | {"L_2 reg. trained with 10^6 data":34} |'

string_RML_e4_0 = f'{"RML-e4-n":15} | {"10^4":12} | {"L_2 (RML)":14} | {"RML model number n with 10^4 data":34} |'
string_RML_e4_mean = f'{"RML-e4-mean":15} | {"":12} | {"":14} | {"Mean of RML-e4-1 to RML-e4-25":34} |'
string_RML_e4_median = f'{"RML-e4-median":15} | {"":12} | {"":14} | {"Median of RML-e4-1 to RML-e4-25":34} |'

string_RML_e5_0 = f'{"RML-e5-n":15} | {"10^5":12} | {"L_2 (RML)":14} | {"RML model number n with 10^5 data":34} |'
string_RML_e5_mean = f'{"RML-e5-mean":15} | {"":12} | {"":14} | {"Mean of RML-e5-1 to RML-e5-25":34} |'
string_RML_e5_median = f'{"RML-e5-median":15} | {"":12} | {"":14} | {"Median of RML-e5-1 to RML-e5-25":34} |'

#string_RML_e6_0 = f'{"RML-e6-0":13} | {"1e6":4} |'
#string_RML_e6_mean = f'{"RML-e6-mean":13} | {"1e6":4} |'
#string_RML_e6_median = f'{"RML-e6-median":13} | {"1e6":4} |'

string_RML_e4_var = f'{"RML-e4-var":15} | {"":12} | {"":14} | {"Variance of RML-e4-1 to RML-e4-25":34} |'
string_RML_e5_var = f'{"RML-e5-var":15} | {"":12} | {"":14} | {"Variance of RML-e5-1 to RML-e5-25":34} |'


string_break = '-'*86
total_string = ""

print(title1); total_string+=title1+'\n'
print(string_break); total_string+=string_break+'\n'
print(string_ML_inf); total_string+=string_ML_inf+'\n'
print(string_ML_inf_reg); total_string+=string_ML_inf_reg+'\n'
#print(string_break); total_string+=string_break+'\n'

print(string_MAP_e4_R100); total_string+=string_MAP_e4_R100+'\n'
print(string_MAP_e5); total_string+=string_MAP_e5+'\n'
print(string_MAP_e6_R100); total_string+=string_MAP_e6_R100+'\n'
#print(string_break); total_string+=string_break+'\n'

print(string_RML_e4_0); total_string+=string_RML_e4_0+'\n'
#print(string_RML_e4_mean); total_string+=string_RML_e4_mean+'\n'
#print(string_RML_e4_median); total_string+=string_RML_e4_median+'\n'
#print(string_break); total_string+=string_break+'\n'

print(string_RML_e5_0); total_string+=string_RML_e5_0+'\n'
#print(string_RML_e5_mean); total_string+=string_RML_e5_mean+'\n'
#print(string_RML_e5_median); total_string+=string_RML_e5_median+'\n'
#print(string_break); total_string+=string_break+'\n'
print(string_break); total_string+=string_break+'\n'

print(title2); total_string+=title2+'\n'
print(string_break); total_string+=string_break+'\n'
print(string_RML_e4_mean); total_string+=string_RML_e4_mean+'\n'
print(string_RML_e4_median); total_string+=string_RML_e4_median+'\n'
print(string_RML_e5_mean); total_string+=string_RML_e5_mean+'\n'
print(string_RML_e5_median); total_string+=string_RML_e5_median+'\n'
print(string_RML_e4_var); total_string+=string_RML_e4_var+'\n'
print(string_RML_e5_var); total_string+=string_RML_e5_var+'\n'
print(string_break); total_string+=string_break+'\n'


Model           | N train data | Regularization | Comment                            |
--------------------------------------------------------------------------------------
ML-inf          | Infinite     | None           | Maximum likelihood model           |
MAP-inf         | Infinite     | L_2            | Model with L_2 regularization      |
MAP-e4          | 10^4         | L_2            | L_2 reg. trained with 10^4 data    |
MAP-e5          | 10^5         | L_2            | L_2 reg. trained with 10^5 data    |
MAP-e6          | 10^6         | L_2            | L_2 reg. trained with 10^6 data    |
RML-e4-n        | 10^4         | L_2 (RML)      | RML model number n with 10^4 data  |
RML-e5-n        | 10^5         | L_2 (RML)      | RML model number n with 10^5 data  |
--------------------------------------------------------------------------------------
Statistic       |              |                | Comment                            |
-------------------------------------------

In [64]:
with open('../figures/plot_table_prediction_error/plot_table_description.txt', 'w') as outfile:
    outfile.write('\n\n' + total_string[:-1])